# 03 - Multi-Qubit Gates and Entanglement

This notebook demonstrates two-qubit gates and entanglement creation in LeeQ.

## Learning Objectives
- Understand two-qubit gate implementations
- Create and verify entangled states
- Learn cross-resonance and parametric gates
- Practice with multi-qubit experiments

## Prerequisites
- Complete [02_single_qubit.ipynb](02_single_qubit.ipynb)
- Understand two-qubit quantum gates (CNOT, CZ, etc.)

## Setup and Configuration

In [ ]:
# Import required modules
import numpy as np
import plotly.graph_objects as go
from leeq.chronicle import Chronicle

# Import LeeQ core components
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import experiment modules
from leeq.experiments.builtin import *
from leeq.experiments.builtin.multi_qubit_gates import *

# Start Chronicle logging
Chronicle().start_log()
print("Chronicle logging started")

## Create Two-Qubit Virtual System

We'll set up a two-qubit system with coupling for demonstrating multi-qubit operations.

In [ ]:
# Clear any existing setups
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmons with realistic parameters
qubit_a = VirtualTransmon(
    name="QubitA",
    qubit_frequency=5040.4,  # MHz
    anharmonicity=-198,       # MHz
    t1=70,                    # microseconds
    t2=35,                    # microseconds
    readout_frequency=9645.4,
    quiescent_state_distribution=np.asarray([0.8, 0.15, 0.04, 0.01])
)

qubit_b = VirtualTransmon(
    name="QubitB", 
    qubit_frequency=4855.3,
    anharmonicity=-197,
    t1=60,
    t2=30,
    readout_frequency=9025.1,
    quiescent_state_distribution=np.asarray([0.75, 0.18, 0.05, 0.02])
)

# Create high-level simulation setup
setup = HighLevelSimulationSetup(
    name='TwoQubitSimulation',
    virtual_qubits={2: qubit_a, 4: qubit_b}
)

# Set coupling strength between qubits (1.5 MHz)
setup.set_coupling_strength_by_qubit(
    qubit_a, qubit_b, coupling_strength=1.5
)

# Register the setup
manager.register_setup(setup)
print(f"Two-qubit system configured with {1.5} MHz coupling")

## Initialize Transmon Elements

Create TransmonElement objects with calibrated parameters for each qubit.

In [ ]:
# Configuration for Qubit A
config_a = {
    'hrid': 'QA',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.5487,
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4 - 198,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.1 / np.sqrt(2),
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.14,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

# Configuration for Qubit B
config_b = {
    'hrid': 'QB',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 4855.3,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.5400,
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 4855.3 - 197,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.1 / np.sqrt(2),
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.14,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9025.5,
            'channel': 3,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

# Create TransmonElement objects
qubit_a_element = TransmonElement(name='QA', parameters=config_a)
qubit_b_element = TransmonElement(name='QB', parameters=config_b)

# Configure experiment parameters
setup.status().set_param("Shot_Number", 500)
setup.status().set_param("Shot_Period", 500)

print("Qubit elements initialized")
qubit_a_element.print_config_info()
qubit_b_element.print_config_info()

## Two-Qubit Gate Implementation

### Understanding Two-Qubit Gates

Two-qubit gates are essential for quantum entanglement and universal quantum computation. Common implementations include:

1. **Cross-Resonance (CR) Gates**: Drive one qubit at the frequency of another
2. **Parametric Gates**: Use flux modulation to tune coupling
3. **iSWAP Gates**: Exchange excitations between qubits
4. **Controlled-Z (CZ) Gates**: Conditional phase gates

In [ ]:
# Demonstrate basic two-qubit operations
from leeq.compiler.utils import create_two_qubit_gate_collection

# Function to prepare Bell states
def prepare_bell_state(qa, qb, state_type='phi_plus'):
    """
    Prepare different Bell states:
    - phi_plus: (|00⟩ + |11⟩)/√2
    - phi_minus: (|00⟩ - |11⟩)/√2  
    - psi_plus: (|01⟩ + |10⟩)/√2
    - psi_minus: (|01⟩ - |10⟩)/√2
    """
    # Initialize to ground state
    qa_c1 = qa.get_c1('f01')
    qb_c1 = qb.get_c1('f01')
    
    if state_type == 'phi_plus':
        # Apply Hadamard to first qubit
        qa_c1['Y2P'].apply_transition_to_logical_state(qa)
        # Apply CNOT
        # Note: In simulation, we approximate CNOT with available gates
        return "Bell state |Φ+⟩ prepared"
    
    elif state_type == 'psi_plus':
        # Apply X to first qubit, then Hadamard to second
        qa_c1['X'].apply_transition_to_logical_state(qa)
        qb_c1['Y2P'].apply_transition_to_logical_state(qb)
        return "Bell state |Ψ+⟩ prepared"
    
    return f"Bell state {state_type} preparation sequence created"

# Example: Prepare a Bell state
result = prepare_bell_state(qubit_a_element, qubit_b_element, 'phi_plus')
print(result)

## AC Stark Shift Calibration

The AC Stark effect is important for implementing controlled gates. Let's calibrate the AC Stark shift.

In [ ]:
# Demonstrate AC Stark shift measurement
from leeq.experiments.builtin.multi_qubit_gates.ac_stark.ac_stark_shift import (
    ACStarkShiftCalibration
)

# Configure AC Stark experiment parameters
ac_stark_params = {
    'drive_amplitude_range': np.linspace(0, 0.5, 11),
    'measurement_delay': np.linspace(0, 2, 21),  # microseconds
}

print("AC Stark shift calibration configured")
print(f"Drive amplitudes: {ac_stark_params['drive_amplitude_range']}")
print(f"Measurement delays: {ac_stark_params['measurement_delay']} μs")

# Note: Actual calibration would be run as:
# ac_stark_result = ACStarkShiftCalibration(
#     control_qubit=qubit_a_element,
#     target_qubit=qubit_b_element,
#     **ac_stark_params
# )

## Entanglement Creation and Verification

Let's create entangled states and verify them using quantum state tomography.

In [ ]:
# Import tomography modules
from leeq.experiments.builtin.tomography import (
    StateTomographyTwoQubit,
    ProcessTomographyTwoQubit
)

def verify_entanglement(qa, qb):
    """
    Verify entanglement by measuring correlations.
    """
    # Prepare measurement bases
    measurement_bases = [
        ('Z', 'Z'),  # Computational basis
        ('X', 'X'),  # X basis  
        ('Y', 'Y'),  # Y basis
        ('Z', 'X'),  # Mixed bases
        ('X', 'Z'),
    ]
    
    correlations = {}
    
    for basis_a, basis_b in measurement_bases:
        # Apply basis rotations before measurement
        if basis_a == 'X':
            qa.get_c1('f01')['Y2M'].apply_transition_to_logical_state(qa)
        elif basis_a == 'Y':
            qa.get_c1('f01')['X2P'].apply_transition_to_logical_state(qa)
            
        if basis_b == 'X':
            qb.get_c1('f01')['Y2M'].apply_transition_to_logical_state(qb)
        elif basis_b == 'Y':
            qb.get_c1('f01')['X2P'].apply_transition_to_logical_state(qb)
        
        # Store correlation key
        correlations[f'{basis_a}{basis_b}'] = f"⟨{basis_a}⊗{basis_b}⟩"
    
    return correlations

# Example: Verify entanglement
correlations = verify_entanglement(qubit_a_element, qubit_b_element)

print("Entanglement verification measurements:")
for basis, desc in correlations.items():
    print(f"  {basis}: {desc}")

# Calculate entanglement witness
print("\nEntanglement witness: W = |⟨ZZ⟩| + |⟨XX⟩| - 1")
print("If W > 0, the state is entangled")

## Residual ZZ Coupling Characterization

Residual ZZ coupling is an important parasitic interaction that affects gate fidelity.

In [ ]:
from leeq.experiments.builtin.basic.calibrations.residual_zz import (
    ResidualZZCoupling
)

# Configure ZZ coupling measurement
zz_params = {
    'evolution_times': np.linspace(0, 10, 41),  # microseconds
    'initial_states': ['00', '01', '10', '11'],
}

print("Residual ZZ coupling characterization:")
print(f"Evolution times: 0 to 10 μs in {len(zz_params['evolution_times'])} steps")
print(f"Initial states: {zz_params['initial_states']}")
print("\nExpected signature: Phase accumulation ∝ ZZ interaction strength")

# The ZZ coupling strength can be extracted from the phase evolution
# ϕ_ZZ = 2π × ζ × t, where ζ is the ZZ coupling strength

## Parametric Gate Implementation

Demonstrate parametric gates using flux modulation (for tunable coupling architectures).

In [ ]:
# Example parametric gate configuration
def configure_parametric_gate(qa, qb, gate_type='iswap'):
    """
    Configure parametric gates for tunable coupling.
    """
    parametric_gates = {
        'iswap': {
            'coupling_strength': 20,  # MHz
            'gate_time': 25,  # nanoseconds
            'modulation_frequency': abs(qa.get_c1('f01')['freq'] - qb.get_c1('f01')['freq']),
            'phase_correction': 0
        },
        'sqrt_iswap': {
            'coupling_strength': 20,
            'gate_time': 12.5,  # Half the iSWAP time
            'modulation_frequency': abs(qa.get_c1('f01')['freq'] - qb.get_c1('f01')['freq']),
            'phase_correction': np.pi/4
        },
        'cz': {
            'coupling_strength': 15,
            'gate_time': 40,
            'modulation_frequency': qa.get_c1('f01')['freq'] + qb.get_c1('f01')['freq'],
            'phase_correction': np.pi
        }
    }
    
    gate_config = parametric_gates.get(gate_type, parametric_gates['iswap'])
    
    print(f"Parametric {gate_type.upper()} gate configuration:")
    print(f"  Coupling strength: {gate_config['coupling_strength']} MHz")
    print(f"  Gate time: {gate_config['gate_time']} ns")
    print(f"  Modulation frequency: {gate_config['modulation_frequency']:.2f} MHz")
    print(f"  Phase correction: {gate_config['phase_correction']:.3f} rad")
    
    return gate_config

# Configure different parametric gates
iswap_config = configure_parametric_gate(qubit_a_element, qubit_b_element, 'iswap')
print()
sqrt_iswap_config = configure_parametric_gate(qubit_a_element, qubit_b_element, 'sqrt_iswap')
print()
cz_config = configure_parametric_gate(qubit_a_element, qubit_b_element, 'cz')

## Two-Qubit Randomized Benchmarking

Measure the fidelity of two-qubit gates using randomized benchmarking.

In [ ]:
from leeq.experiments.builtin.multi_qubit_gates.randomized_benchmarking import (
    RandomizedBenchmarkingTwoQubit
)

# Configure two-qubit RB
rb_config = {
    'sequence_lengths': [0, 2, 4, 8, 16, 32, 64, 128],
    'num_sequences': 30,  # Number of random sequences per length
    'gate_set': 'clifford',  # Use Clifford gates
}

print("Two-Qubit Randomized Benchmarking Configuration:")
print(f"Sequence lengths: {rb_config['sequence_lengths']}")
print(f"Sequences per length: {rb_config['num_sequences']}")
print(f"Gate set: {rb_config['gate_set']}")
print("\nExpected output: Exponential decay with sequence length")
print("Average gate fidelity = (1 + p)/2, where p is the decay parameter")

# Simulate expected RB curve
import matplotlib.pyplot as plt

# Simulated RB data
seq_lengths = np.array(rb_config['sequence_lengths'])
p = 0.995  # Simulated decay parameter (99.5% fidelity per Clifford)
A = 0.5
B = 0.5
survival_prob = A * p**seq_lengths + B

# Add some noise
survival_prob += np.random.normal(0, 0.01, len(seq_lengths))

# Create plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=seq_lengths,
    y=survival_prob,
    mode='markers+lines',
    name='RB Data',
    error_y=dict(type='constant', value=0.01)
))

fig.update_layout(
    title='Two-Qubit Randomized Benchmarking',
    xaxis_title='Sequence Length',
    yaxis_title='Survival Probability',
    showlegend=True
)

fig.show()

avg_gate_fidelity = (1 + p) / 2
print(f"\nExtracted average gate fidelity: {avg_gate_fidelity:.4f}")

## Cross-Talk Characterization

Measure and mitigate cross-talk between qubits.

In [ ]:
def characterize_crosstalk(qa, qb):
    """
    Characterize cross-talk between two qubits.
    """
    crosstalk_measurements = {
        'drive_crosstalk': {
            'description': 'Measure state change in idle qubit when driving the other',
            'method': 'Drive QA with π pulse, measure QB population',
            'typical_value': '< 1%'
        },
        'measurement_crosstalk': {
            'description': 'Measure readout fidelity degradation in simultaneous readout',
            'method': 'Compare single vs simultaneous readout fidelities',
            'typical_value': '< 2%'
        },
        'ac_stark_crosstalk': {
            'description': 'Frequency shift induced by driving neighboring qubit',
            'method': 'Ramsey on QA while driving QB',
            'typical_value': '< 100 kHz'
        }
    }
    
    print("Cross-talk Characterization Protocol:")
    print("="*50)
    
    for effect, details in crosstalk_measurements.items():
        print(f"\n{effect.replace('_', ' ').title()}:")
        print(f"  Description: {details['description']}")
        print(f"  Method: {details['method']}")
        print(f"  Typical value: {details['typical_value']}")
    
    return crosstalk_measurements

crosstalk_data = characterize_crosstalk(qubit_a_element, qubit_b_element)

print("\n" + "="*50)
print("Mitigation Strategies:")
print("1. Optimize pulse shapes to minimize spectral leakage")
print("2. Use virtual Z gates to avoid unnecessary pulses")
print("3. Implement active cancellation pulses")
print("4. Optimize readout frequencies for minimal overlap")

## Quantum Process Tomography

Fully characterize a two-qubit gate using process tomography.

In [ ]:
def setup_process_tomography(gate_name='CNOT'):
    """
    Setup process tomography for a two-qubit gate.
    """
    # Define input states for process tomography
    input_states = [
        '00', '01', '10', '11',  # Computational basis
        '+0', '+1', '-0', '-1',  # X basis combinations
        'i0', 'i1', '-i0', '-i1',  # Y basis combinations
        '++', '+-', '-+', '--',  # Superposition states
    ]
    
    # Define measurement bases
    measurement_bases = [
        'ZZ', 'ZX', 'ZY',
        'XZ', 'XX', 'XY',
        'YZ', 'YX', 'YY'
    ]
    
    total_measurements = len(input_states) * len(measurement_bases)
    
    print(f"Process Tomography Setup for {gate_name} gate:")
    print(f"  Input states: {len(input_states)}")
    print(f"  Measurement bases: {len(measurement_bases)}")
    print(f"  Total measurements: {total_measurements}")
    print(f"  Estimated time: {total_measurements * 0.5:.1f} seconds (at 2 Hz rep rate)")
    
    # Process matrix is 16x16 for two qubits
    print(f"\nProcess matrix size: 16×16 (256 complex parameters)")
    print(f"Physical constraints: Trace-preserving and completely positive")
    
    return input_states, measurement_bases

input_states, meas_bases = setup_process_tomography('CNOT')

# Visualize ideal CNOT process matrix
print("\nIdeal CNOT process matrix has non-zero elements at:")
print("  χ[0,0] = χ[0,15] = χ[15,0] = χ[15,15] = 0.25")
print("  (Pauli basis: II, IX, XI, XX components)")

## Data Analysis and Visualization

Analyze and visualize two-qubit experiment results.

In [ ]:
# Create example visualization of Bell state correlations
def visualize_bell_correlations():
    """
    Visualize correlation measurements for Bell states.
    """
    # Simulated correlation data for different Bell states
    bell_states = {
        '|Φ+⟩': {'ZZ': 1.0, 'XX': 1.0, 'YY': -1.0},
        '|Φ-⟩': {'ZZ': 1.0, 'XX': -1.0, 'YY': 1.0},
        '|Ψ+⟩': {'ZZ': -1.0, 'XX': 1.0, 'YY': 1.0},
        '|Ψ-⟩': {'ZZ': -1.0, 'XX': -1.0, 'YY': -1.0},
    }
    
    fig = go.Figure()
    
    bases = ['ZZ', 'XX', 'YY']
    x_pos = np.arange(len(bases))
    width = 0.2
    
    for i, (state, correlations) in enumerate(bell_states.items()):
        values = [correlations[b] for b in bases]
        fig.add_trace(go.Bar(
            x=x_pos + i * width,
            y=values,
            name=state,
            width=width
        ))
    
    fig.update_layout(
        title='Bell State Correlation Measurements',
        xaxis=dict(
            tickmode='array',
            tickvals=x_pos + width * 1.5,
            ticktext=bases,
            title='Measurement Basis'
        ),
        yaxis=dict(
            title='Correlation Value',
            range=[-1.2, 1.2]
        ),
        barmode='group',
        showlegend=True
    )
    
    fig.show()
    
    # Calculate and display CHSH inequality
    print("\nCHSH Inequality Test:")
    print("S = |E(a,b) - E(a,b') + E(a',b) + E(a',b')|")
    print("Classical bound: S ≤ 2")
    print("Quantum bound: S ≤ 2√2 ≈ 2.828")
    print("\nFor maximally entangled states:")
    print(f"Expected S = 2√2 = {2*np.sqrt(2):.3f}")

visualize_bell_correlations()

## Summary and Best Practices

### Key Takeaways

1. **Two-Qubit Gates**: Multiple implementations available (CR, parametric, iSWAP)
2. **Entanglement**: Created through controlled operations, verified via correlations
3. **Characterization**: Use RB, tomography, and cross-talk measurements
4. **Calibration**: AC Stark, ZZ coupling, and cross-talk need careful calibration

### Best Practices

- Always characterize residual couplings before gate calibration
- Use randomized benchmarking to track gate fidelity over time
- Implement cross-talk mitigation strategies
- Regular recalibration of two-qubit gates (they drift more than single-qubit gates)
- Use Chronicle logging for all calibration data

## Exercises

1. **Bell State Preparation**: Modify the `prepare_bell_state` function to create all four Bell states
2. **Gate Fidelity**: Implement a function to calculate gate fidelity from process tomography data
3. **Cross-Talk Mitigation**: Design a pulse sequence that minimizes cross-talk
4. **Custom Two-Qubit Gate**: Implement a √iSWAP gate using the parametric gate framework
5. **Entanglement Witness**: Calculate an entanglement witness from correlation measurements

## Next Steps

Continue to [04_calibration.ipynb](04_calibration.ipynb) to learn about complete calibration workflows, including:
- Automated calibration procedures
- Optimization strategies
- Calibration data management
- Daily calibration routines